# Anisotropic Mg tracer calculation

In [5]:
import numpy as np
from onsager import OnsagerCalc
from onsager import crystal
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.constants import physical_constants
kB = physical_constants['Boltzmann constant in eV/K'][0]
#print (kB)

In [6]:
a=3.18957309719204
c_a=1.6264299717255
HCP = crystal.Crystal.HCP(a0=a,c_a=c_a, chemistry="Mg")
print(HCP)

#Lattice:
  a1 = [ 1.59478655 -2.76225133  0.        ]
  a2 = [ 1.59478655  2.76225133  0.        ]
  a3 = [ 0.          0.          5.18761728]
#Basis:
  (Mg) 0.0 = [ 0.33333333  0.66666667  0.25      ]
  (Mg) 0.1 = [ 0.66666667  0.33333333  0.75      ]


In [7]:
sitelist = HCP.sitelist(0)
#print (HCP.sitelist(0))
#print(sitelist)
vacancyjumps = HCP.jumpnetwork(0, 1.01*a)
for jlist in vacancyjumps:
    print('---')
    for (i,j), dx in jlist:
        print(i, '-', j, dx,np.linalg.norm(dx))
#sitelist

---
1 - 1 [ 1.59478655  2.76225133  0.        ] 3.18957309719
1 - 1 [-1.59478655 -2.76225133 -0.        ] 3.18957309719
0 - 0 [-1.59478655  2.76225133  0.        ] 3.18957309719
0 - 0 [ 1.59478655 -2.76225133 -0.        ] 3.18957309719
0 - 0 [  3.18957310e+00  -1.11022302e-16   0.00000000e+00] 3.18957309719
0 - 0 [ -3.18957310e+00   1.11022302e-16  -0.00000000e+00] 3.18957309719
1 - 1 [ 1.59478655 -2.76225133  0.        ] 3.18957309719
1 - 1 [-1.59478655  2.76225133 -0.        ] 3.18957309719
0 - 0 [-1.59478655 -2.76225133  0.        ] 3.18957309719
0 - 0 [ 1.59478655  2.76225133 -0.        ] 3.18957309719
1 - 1 [ -3.18957310e+00  -1.11022302e-16   0.00000000e+00] 3.18957309719
1 - 1 [  3.18957310e+00   1.11022302e-16  -0.00000000e+00] 3.18957309719
---
1 - 0 [ 0.          1.84150089  2.59380864] 3.18103265953
0 - 1 [-0.         -1.84150089 -2.59380864] 3.18103265953
1 - 0 [-1.59478655 -0.92075044 -2.59380864] 3.18103265953
0 - 1 [ 1.59478655  0.92075044  2.59380864] 3.18103265953
0 - 

In [8]:
HCPdiffuser = OnsagerCalc.VacancyMediated(HCP, 0, sitelist, vacancyjumps, 1)
#print (HCPdiffuser)
#HCPdiffuser.omegalist()

In [9]:
len(HCPdiffuser.interactlist())
for state in HCPdiffuser.interactlist():
    print(state)

1.[0,0,0]:0.[0,0,1] (dx=[0.0,1.8415008862638127,2.5938086411412327])
1.[0,0,0]:1.[1,0,0] (dx=[1.5947865485960198,-2.7622513293957187,0.0])


In [10]:
HCPtracer = {'preV': np.array([1.0/1.1075157070506694]), 'eneV': np.array([0.0]), 
             'preT0': np.array([129.2186479/36.7840204101001, 129.2186479/34.5772672908345]), 'eneT0': np.array([1.211, 1.23])}
HCPtracer.update(HCPdiffuser.maketracerpreene(**HCPtracer))
#for k,v in zip(HCPtracer.keys(), HCPtracer.values()): print(k,v)

Anisotropic diffusivity above 670K from Onsager calculation

In [11]:
Temp=np.arange(700, 923, 100)
D_Onsager=[]
for T in Temp:
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPtracer))
    Lss=Lss*pre
    D_Onsager.append([Lss[0,0],Lss[1,1],Lss[2,2]])
    print(T, Lss[0,0],Lss[1,1],Lss[2,2])
D_Onsager=np.array(D_Onsager)

700 1.10052185758e-15 1.10052185758e-15 9.31208798562e-16
800 1.36793807724e-14 1.36793807724e-14 1.18702281077e-14
900 9.71325832622e-14 9.71325832622e-14 8.59456513209e-14


Below 670 K, code is giving error and the error is:

In [12]:
for T in range(600, 923, 100):
    pre=1e-8 # THz and Angstrom unit scaling
    Lvv, Lss, Lsv, L1vv = HCPdiffuser.Lij(*HCPdiffuser.preene2betafree(kB*T, **HCPtracer))
    Lss=Lss*pre
    print(T, Lss[0,0],Lss[1,1],Lss[2,2])


IndexError: list index out of range

# Analytical tracer diffusion result

In [13]:
#Correlations from Koiwa and Ishioka’s
def fz(x):
    return((1+8.77*x+7.37*x**2+0.65*x**3)/(1+9.90*x+10.9*x**2+x**3))
def fx(x):
    return((0.56+4.38*x+3.67*x**2+0.64*x**3)/(1+5.75*x+4.11*x**2+x**3))

In [14]:
def Basal_rate(T):
    prefactor=(129.2186479/36.78402041)
    mb=0.397
    return(prefactor*np.exp(-mb/(kB*T)))
def c_rate(T):
    prefactor=(129.2186479/34.57726729)
    mc=0.416
    return(prefactor*np.exp(-mc/(kB*T)))

In [15]:
def Vconc(T):
    entropy_contribution=1.1075157070506694
    formationE=0.814228281
    prob=entropy_contribution*np.exp(-formationE/(kB*T))
    return(prob)

In [16]:
def AnalyticalD(T):
    wb=Basal_rate(T)
    wc=c_rate(T)
    x=wb/wc
    c=a*c_a
    Dxx=0.5*Vconc(T)*a**2*fx(x)*(3*wb+wc)
    Dzz=0.75*Vconc(T)*c**2*fz(x)*wc
    return(np.array([Dxx,Dxx,Dzz]))

In [17]:
D_Analytical=[]
for T in Temp:
    D=AnalyticalD(T)*10**-8 #Unit scaling
    print(T, D[0],D[1],D[2])
    D_Analytical.append([D[0],D[1],D[2]])
D_Analytical=np.array(D_Analytical)

700 1.11906625928e-15 1.11906625928e-15 8.85136328873e-16
800 1.38691522973e-14 1.38691522973e-14 1.13678882112e-14
900 9.82542469898e-14 9.82542469898e-14 8.27912864405e-14


Comparison of Onsager and analyitcal result, printing ratio of Onsager D/ Analytical D, Both agree within 5% of their relative values.

In [18]:
Ratio=D_Onsager/D_Analytical
for i,T in enumerate(Temp):
    print (T, Ratio[i][0],Ratio[i][1],Ratio[i][2])

700 0.983428683016 0.983428683016 1.0520512696
800 0.986317006204 0.986317006204 1.04418937688
900 0.988584068761 0.988584068761 1.03810020373


KMC result for three temperatures, statisical error in KMC values is ~0.05%

In [20]:
D_KMC=np.zeros((3,3))
D_KMC[0]=np.array([1.10228325e-15,   1.10133965e-15, 9.30127592e-16])
D_KMC[1]=np.array([1.36918534e-14,   1.37001681e-14,     1.18546593e-14])
D_KMC[2]=np.array([9.73279952e-14,   9.73412524e-14,  8.58637944e-14])

Comparison of Onsager and KMC result, printing ratio of Onsager D/ KMC D, Both agree within 1% of their relative values.

In [21]:
Ratio=D_Onsager/D_KMC
for i,T in enumerate(Temp):
    print (T, Ratio[i][0],Ratio[i][1],Ratio[i][2])

700 0.998402050993 0.999257456658 1.00116242822
800 0.999089047537 0.998482695437 1.00131330706
900 0.997992232991 0.997856313406 1.00095333454
